In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(sys.path[0]+'/../')
sys.path.append(sys.path[0]+'/../utils')

import torch
import torch.nn as nn
import time
from tqdm import tqdm
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from dataloaders.minc_dataloader import MINCDataset, MINCDataLoader
from utils import CosineDecayLR, Metrics

from torchvision.models import swin_v2_b, Swin_V2_B_Weights


In [2]:
minc_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc'
labels_path = '/home/ahmed/workspace/notebook/matrec/datasets/minc/train.txt'
labels_path_t = '/home/ahmed/workspace/notebook/matrec/datasets/minc/test.txt'
checkpoint = "../weights/swin_v2b_minc.pth"

BATCH_SIZE = 8
TRAIN_ITER = 2000
TEST_ITER = 400
size = 256

LOAD = True
lr = 4e-5
start = 58000

train_loader = MINCDataLoader(minc_path, labels_path, batch_size=BATCH_SIZE, size=size, f=0.16)
test_loader = DataLoader(dataset=MINCDataset(minc_path, labels_path_t, size=(size, size)), 
                         batch_size=BATCH_SIZE, num_workers=0, pin_memory=False, shuffle=True)

In [3]:
model = swin_v2_b(weights=Swin_V2_B_Weights.IMAGENET1K_V1)

if LOAD:
    model.load_state_dict(torch.load(checkpoint), strict=False)
model = model.train()
model = model.cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-8)
lr_scheduler = CosineDecayLR(optimizer, lr, 540000)
loss = Metrics()

In [4]:
#One epoch done
for epc in range(2, 3):
    ticket = "Epoch {} starting from iteration {}: ".format(epc, start)
    
    r = tqdm(range(start, len(train_loader)), leave=False, desc=ticket, total=len(train_loader)-start)    
    for idx in r:
        x, y = train_loader[idx]
        y_pred = model(x.cuda())
        lf = loss.compute(y_pred, y)
        lf.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        optimizer.zero_grad()
        r.set_postfix(loss=lf.item())
        
        if idx % TRAIN_ITER == 0:
            #save checkpoint
            if idx != start:
                torch.save(model.state_dict(), checkpoint)
            #decrease lr
            lr = lr_scheduler.step(epc * 180000 + start + idx)
            #test loss run on val
            with torch.no_grad():
                ac = 0
                for i in range(TEST_ITER):
                    x, y = next(iter(test_loader))
                    y_pred = model(x.cuda())
                    ac = ac + loss.accuracy(y_pred, y)
                ac = ac/TEST_ITER
                print("Iteration:", idx,
                    "\nTest N", str(epc) + '-' + str(idx//TRAIN_ITER) + ' : ' + str(float(ac)),
                     "\nLearning rate:", lr,
                     )
    start = 0
    torch.save(model.state_dict(), checkpoint)

Epoch 2 starting from iteration 58000:   0%|          | 1/120251 [01:56<3905:18:08, 116.92s/it, loss=0.135]

Iteration: 58000 
Test N 2-29 : 78.34375 
Learning rate: 1.3704052214771513e-06


Epoch 2 starting from iteration 58000:   2%|▏         | 2001/120251 [17:52<1181:49:38, 35.98s/it, loss=1.27]  

Iteration: 60000 
Test N 2-30 : 79.75 
Learning rate: 1.2870096832093726e-06


Epoch 2 starting from iteration 58000:   3%|▎         | 4001/120251 [33:40<1149:58:33, 35.61s/it, loss=0.118]  

...

Epoch 2 starting from iteration 58000:  62%|██████▏   | 74001/120251 [9:47:30<477:20:00, 37.15s/it, loss=0.0687]

Iteration: 132000 
Test N 2-66 : 79.625 
Learning rate: 3.383683457463649e-08


Epoch 2 starting from iteration 58000:  63%|██████▎   | 76001/120251 [10:03:58<457:08:10, 37.19s/it, loss=0.00391]

Iteration: 134000 
Test N 2-67 : 79.375 
Learning rate: 4.871899480351605e-08


KeyboardInterrupt: 